In [ ]:

from transformers import pipeline
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sys
sys.path.append("../CODE-Baseline")  
import warnings
warnings.filterwarnings('ignore')


from salary_baseline import extract_salary_with_inference


file_path = '../DATASETS/salary_labelled_development_set.csv'
test_file_path = '../DATASETS/salary_labelled_test_set.csv'
df = pd.read_csv(file_path)

model_name = "deepset/roberta-base-squad2"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
# html -> context 
def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    for tag in soup(["script", "style"]):
        tag.decompose()
    text = soup.get_text(separator=" ", strip=True)
    return text

In [8]:
salary_question = "what is the salary?"
pay_freq_question = "Is the salary paid monthly, hourly, yearly, weekly, or daily?"

def get_salary_using_nFT_Bert(text,nation_code):
  context=clean_html_tags(text)
  salary_result = qa_pipeline({"context": context, "question": salary_question})
  
  t_result=extract_salary_with_inference(salary_result["answer"],nation_code)
  # print(t_result)
  if t_result!='0-0-None-None':
    pay_freq_result = qa_pipeline({"context": context, "question": pay_freq_question})
    # print(pay_freq_result["answer"])
    
  return t_result

    
    
    
df['predicted_salary'] = df.apply(
    lambda row: get_salary_using_nFT_Bert(
        f"{row['job_title']} {row['job_ad_details']}",
        row['nation_short_desc']
    ),
    axis=1
)
    


In [9]:
# TP, FP, TN, FN
TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))

precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

# Print prediction vs ground truth
# print("\n🔍 Prediction vs Ground Truth:\n")
# for i, row in df.iterrows():
#     predicted = row['predicted_salary']
#     expected = row['y_true']
#     if predicted != expected:
#         print(f"[{i}] ❌ Predicted: {predicted} | Expected: {expected}")
#         print(f"{row['job_id']} {row['job_title']} {row['job_ad_details']}")
#         print()
    # else:
    #     print(f"[{i}] ✅ Matched:   {predicted}")

print("Development dataset:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print()

Development dataset:
Precision: 0.7220
Recall: 0.7746
F1 Score: 0.7474
Accuracy: 0.7653



In [10]:
# test set
df = pd.read_csv(test_file_path)
df['predicted_salary'] = df.apply(
    lambda row: get_salary_using_nFT_Bert(
        f"{row['job_title']} {row['job_ad_details']}",
        row['nation_short_desc']
    ),
    axis=1
)

TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))


precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
accuracy = (TP + TN) / (FP + FN + TP + TN)

print("Test dataset:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Test dataset:
Precision: 0.6903
Recall: 0.7520
F1 Score: 0.7198
Accuracy: 0.7460
